In [ ]:
# Read names of the mocules
names_dict = {}
with open("data/names_ids_only2.txt", "r") as file:
    for line in file:
        elements = line.split(" ")
        id = elements[-1]
        name = "".join(elements[:-1])
        name = name.strip()
        id = id.strip()
        names_dict[id] = name    

In [ ]:
# Get SMILES from OPSIN
import requests
from rdkit import Chem

def name_to_mol(name):
    url = f"https://opsin.ch.cam.ac.uk/opsin/{name}.smi"
    response = requests.get(url)
    if response.status_code == 200:
        smiles = response.text.strip()
        return smiles
    else:
        return None

smiles_dict = {}
for id, name in names_dict.items():
    smiles = name_to_mol(name)
    smiles_dict[id] = smiles

# Remove NAs
smiles_dict = {id: smiles for id, smiles in smiles_dict.items() if smiles is not None}

In [ ]:
# Append SMILES and IDs to the file previous generated by the R code
with open("data/smiles_ids.txt", "a") as file:
    for id, smiles in smiles_dict.items():
        file.write(f"{smiles} {id}\n")